# Capstone Project  
### Introduction/Business Problem  
#### Tempe, Arizona is ranked 11th most bike friendly city in the United States. The city has dedicated bike lanes. The bike traffic is most concentrated near the Arizona State University, Tempe campus.  

#### What I will try to analyze is whether it would make sense to open a new bike store in Tempe in order to meet the rising demand for bikes in the city?

### Data Section  
#### 1. I will be using the Foursqaure location data to analyze this problem. First I will pull 10000 venues in a 100 KM radius of Tempe, Arizona.  
#### 2. The Foursquare API will provide me with the all the information related to the venues.  
#### 3. From this data, I will select columns venue id, venue name, venue categories, venue latitude and venue longitude
#### 4. Then by applying the bike store filter, I will check how many bike stores are in Tempe
#### 5. Plot the stores on the map of Tempe, AZ
#### 6. Get the ratings for all the stores
#### 7. Make a recommendation based on the analysis

In [1]:
#Import all the packages needed for the analysis

from geopy.geocoders import Nominatim
import folium
import pandas as pd
import numpy as np
import json
import bs4 as BeautifulSoup
import requests
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [2]:
#Using the geopy package get the latitude and longitude for Tempe, Arizona

address = "Tempe, AZ"

geolocator = Nominatim(user_agent='tempe_agent')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print(latitude,longitude)

33.4255056 -111.9400125


In [3]:
#Store the FourSquare credentials needed to access the API in variables.
CLIENT_ID = 'BKHRG22YXC4QHDV2AMOKHJAYO2GOXF2LTSASSGUEY1GWTGXY' # your Foursquare ID
CLIENT_SECRET = 'MHLYUPXJEN4EDEOJ5TGLGVAOWRJURI2JM0AGF41BGD43JYXF' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

#Set a 100 KM radius and a limit of 10000 results from the query
RADIUS = 1000
LIMIT = 10000
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: BKHRG22YXC4QHDV2AMOKHJAYO2GOXF2LTSASSGUEY1GWTGXY
CLIENT_SECRET:MHLYUPXJEN4EDEOJ5TGLGVAOWRJURI2JM0AGF41BGD43JYXF


In [4]:
#Build the url that will be used to request data from the FourSquare API

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&limit={}&radius={}'.format(CLIENT_ID,CLIENT_SECRET,VERSION,latitude,longitude,LIMIT,RADIUS)
print(url)

https://api.foursquare.com/v2/venues/explore?client_id=BKHRG22YXC4QHDV2AMOKHJAYO2GOXF2LTSASSGUEY1GWTGXY&client_secret=MHLYUPXJEN4EDEOJ5TGLGVAOWRJURI2JM0AGF41BGD43JYXF&v=20180605&ll=33.4255056,-111.9400125&limit=10000&radius=1000


In [5]:
#Request the data from the FourSquareAPI 

results = requests.get(url).json()

In [6]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [7]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.id','venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,lat,lng
0,4decfb68ae60a6bd3895c4bc,Daily Jam,American Restaurant,33.427191,-111.940375
1,540e37c9498e8f0de59817ba,Ike's Love & Sandwiches,Sandwich Place,33.423823,-111.939404
2,4a42d5e9f964a5204aa61fe3,Rúla Búla Irish Pub and Restaurant,Irish Pub,33.426455,-111.939862
3,4eb0eb14003940a4cd368b67,Slickables,Dessert Shop,33.423444,-111.939999
4,4a8f2648f964a5201d1420e3,House of Tricks,New American Restaurant,33.423443,-111.937456


In [8]:
#From the nearby_venues dataframe isolate the bike stores

bike_store = nearby_venues[nearby_venues['categories'].str.contains('Bike')]

In [9]:
#Plot the location of the Bike Store on the map of Tempe Arizona using the Folium package in Python

bike_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Tempe, Arizona
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Tempe, AZ',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(bike_map)

# add the bike stores as blue circle markers
for lat, lng, label in zip(bike_store.lat, bike_store.lng, bike_store.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(bike_map)

# display map
bike_map

In [10]:
bike_store

,id,name,categories,lat,lng
6,4c49b8373013a593566720e3,The Bicycle Cellar,Bike Shop,33.425728,-111.936436


In [11]:
bike_id = '4c49b8373013a593566720e3' # ID of The Bicycle Cellar

#Use the id of the Bicycle Cellar to build the url that will get additional information of the venue
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(bike_id, CLIENT_ID, CLIENT_SECRET, VERSION)
url

'https://api.foursquare.com/v2/venues/4c49b8373013a593566720e3?client_id=BKHRG22YXC4QHDV2AMOKHJAYO2GOXF2LTSASSGUEY1GWTGXY&client_secret=MHLYUPXJEN4EDEOJ5TGLGVAOWRJURI2JM0AGF41BGD43JYXF&v=20180605'

In [12]:
#Request data about the venue using the url built in the preious step

result = requests.get(url).json()
print(result['response']['venue'].keys())
result['response']['venue']

dict_keys(['id', 'name', 'contact', 'location', 'canonicalUrl', 'categories', 'verified', 'stats', 'url', 'likes', 'dislike', 'ok', 'rating', 'ratingColor', 'ratingSignals', 'allowMenuUrlEdit', 'beenHere', 'specials', 'photos', 'venuePage', 'reasons', 'description', 'storeId', 'page', 'hereNow', 'createdAt', 'tips', 'shortUrl', 'timeZone', 'listed', 'hours', 'popular', 'seasonalHours', 'defaultHours', 'pageUpdates', 'inbox', 'attributes', 'bestPhoto', 'colors'])


{'id': '4c49b8373013a593566720e3',
 'name': 'The Bicycle Cellar',
 'contact': {'phone': '4802197225',
  'formattedPhone': '(480) 219-7225',
  'twitter': 'bicyclecellar',
  'facebook': '121399153246',
  'facebookUsername': 'thebicyclecellar',
  'facebookName': 'The Bicycle Cellar'},
 'location': {'address': '200 E 5th St Ste 105',
  'crossStreet': '5th Street and College',
  'lat': 33.42572829789885,
  'lng': -111.93643640883742,
  'labeledLatLngs': [{'label': 'display',
    'lat': 33.42572829789885,
    'lng': -111.93643640883742},
   {'label': 'entrance', 'lat': 33.425326, 'lng': -111.936174}],
  'postalCode': '85281',
  'cc': 'US',
  'city': 'Tempe',
  'state': 'AZ',
  'country': 'United States',
  'formattedAddress': ['200 E 5th St Ste 105 (5th Street and College)',
   'Tempe, AZ 85281',
   'United States']},
 'canonicalUrl': 'https://foursquare.com/v/the-bicycle-cellar/4c49b8373013a593566720e3',
 'categories': [{'id': '4bf58dd8d48988d115951735',
   'name': 'Bike Shop',
   'pluralNa

In [13]:
#Check the rating of the store 

try:
    print(result['response']['venue']['rating'])
except:
    print('This venue has not been rated yet.')

8.7


## Result/Recommendation:  
#### Even though the rating for the bike store is good, we can see that there is only one store serving the entire city of Tempe. Considering the bike friendly nature of the city and the growing student population, once store cannot serve the entire city.  
### Hence I would recommend opening a new bike store near the Arizona State University, Tempe Campus.